In [1]:
# This function checks the quality of split by calcultaing the gini index
def gini_index(groups, classes):
    # Count all samples of all the groups
    samples = float(sum([len(group) for group in groups]))
    # Initialize the gini value to 0.0
    gini = 0.0
    for group in groups:
        #Take size of every group 
        size = float(len(group))
        # Avoid divide by zero if size is zero
        if size == 0: 
            continue
        score = 0.0
        #find out proportion of the class in group
        for class_val in classes:
            proportion = [row[-1] for row in group].count(class_val)/size
            #print(proportion)
            score += proportion*proportion
        #Update the gini score
        gini += (1-score)*size/samples
    return gini
# Validating the function
#print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
#print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))

In [2]:
# This function splts the dataset into two groups based on given parameters
# Such as index , value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index]<value:
            left.append(row)
        else: 
            right.append(row)
    return left, right

In [39]:
# This function find out the point of split by hit and trail method , cheking all possible
#combinations over the dataset
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    #print(class_values)
    b_index, b_value, b_score, groups = 999, 999, 999, None
    for index in range(len(class_values)):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            #print(index, gini)
            if gini<b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index': b_index, 'value': b_value, 'groups': b_groups }
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
split = get_split(dataset)
print('The index and value  of the attribute we get the best split: ', split['index'], split['value'])

The index and value  of the attribute we get the best split:  0 6.642287351


Terminal node :
   decide when to stop growing tree.
   Max tree depth:
       if tree has reached to the max depth from the root, then we have to stop further spliting.
       
   Min Node recordes: 
       if we have min records for spliting the we have to stop spliting the node.
       
   All belong to same Class:
       If all the rows belong to same class then we also have to stop the division.
   

In [4]:
# Specifies the class to terminal node further used for classification.
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key = outcomes.count)


In [42]:
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for null left or right node
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    #process left child
    if len(left)<= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    #process right child
    if len(right)<= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

In [43]:
def build_tree(train_data, max_depth, min_size):
    root = get_split(train_data)
    split(root, max_depth, min_size, 1)
    return root

In [8]:
def print_tree(node, depth = 0):
    if isinstance(node, dict):
        print("%s[X%d < %.3f]"%((depth*'', (node['index']+1), node['value'])) )
        print_tree(node['left'])
        print_tree(node['right'])
    else:
        print("%s[%s]"%((depth*' ', node)))
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
tree = build_tree(dataset, 3, 1)
print_tree(tree)

[0, 1]
0 0.4444444444444445
0 0.5
0 0.2857142857142857
0 0.1666666666666666
0 0.375
0 0.2857142857142857
0 0.375
0 0.1666666666666666
0 0.4444444444444445
0 0.0
1 0.5
1 0.4444444444444445
1 0.31999999999999984
1 0.41666666666666663
1 0.4761904761904763
1 0.1666666666666666
1 0.4444444444444445
1 0.5
1 0.2857142857142857
1 0.375
[0]
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
[0]
0 0.0
0 0.0
0 0.0
0 0.0
[1]
0 0.0
0 0.0
0 0.0
0 0.0
0 0.0
[1]
0 0.0
0 0.0
[1]
0 0.0
0 0.0
0 0.0
[X1 < 6.642]
[X1 < 2.771]
[0]
[X1 < 2.771]
[0]
[0]
[X1 < 7.498]
[X1 < 7.445]
[1]
[1]
[X1 < 7.498]
[1]
[1]


Below Function is used to predict the values from the given tree and row which we want to predict.


In [13]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
    
stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0}
for row in dataset:
    prediction = predict(stump, row)
    print('Expected = %d, Got = %d'% (row[-1], prediction))

Expected = 0, Got = 0
Expected = 0, Got = 0
Expected = 0, Got = 0
Expected = 0, Got = 0
Expected = 0, Got = 0
Expected = 1, Got = 1
Expected = 1, Got = 1
Expected = 1, Got = 1
Expected = 1, Got = 1
Expected = 1, Got = 1


In [26]:
#Load the CSV file
from random import seed
from random import randrange
from csv import reader

def load_csv(file_name):
    file = open(file_name, 'rt')
    lines = reader(file)
    dataset = list(lines)
    return dataset
print('vvt')

vvt


In [28]:
#Covert string values in column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [36]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int (len(dataset)/ n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold)<fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [54]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual))*100.0
              

In [55]:
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list(fold)
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores


In [57]:
# Classification and regression tree algortihm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return (predictions)

seed(1)
file_name = "bank_note.csv"
dataset = load_csv(file_name)
dataset = dataset[1:]
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
n_folds = 5
max_depth = 5
min_size = 10

scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('accuracy for every fold:',scores)

accuracy for every fold: [92.7007299270073, 92.33576642335767, 92.7007299270073, 91.97080291970804, 95.25547445255475]
